# Notebook 4: Knowledge Base Assistant

**User Stories**: US4 (Natural-Language CRM Query) + US3 (KB Search)  
**Persona**: Customer Service Representative (CSR)  

Ask natural-language questions against the Salesforce Knowledge Base
and get accurate answers with article citations.

In [ ]:
# Cell 2: Environment + Auth Setup
import os
from pathlib import Path

from dotenv import load_dotenv

env_path = Path("../.env")
if env_path.exists():
    load_dotenv(env_path)

required_vars = [
    "AZURE_AI_PROJECT_ENDPOINT",
    "AZURE_OPENAI_DEPLOYMENT",
    "SF_INSTANCE_URL",
    "SF_ACCESS_TOKEN",
]
missing = [v for v in required_vars if not os.environ.get(v)]
if missing:
    raise OSError(f"Missing required environment variables: {missing}")

print("Environment configured successfully.")
print(f"  Azure AI endpoint: {os.environ['AZURE_AI_PROJECT_ENDPOINT'][:40]}...")
print(f"  Salesforce instance: {os.environ['SF_INSTANCE_URL']}")

In [ ]:
# Cell 3: Start MCP Server + Configure Connection
import subprocess
import sys
import time

from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import MCPTool
from azure.identity import DefaultAzureCredential

project_root = os.path.abspath("..")

# Start salesforce-knowledge MCP server (SSE)
KB_PORT = 8104
kb_process = subprocess.Popen(
    [sys.executable, "-m", "mcp_servers.salesforce_knowledge.server"],
    env={**os.environ, "MCP_TRANSPORT": "sse", "FASTMCP_PORT": str(KB_PORT)},
    cwd=project_root,
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
)

time.sleep(3)

if kb_process.poll() is not None:
    stderr = kb_process.stderr.read().decode() if kb_process.stderr else ""
    raise RuntimeError(f"Knowledge MCP server failed to start: {stderr}")

print(f"✅ Knowledge MCP server started (PID: {kb_process.pid}) at http://127.0.0.1:{KB_PORT}/sse")

# Create AI Project client
project_client = AIProjectClient(
    endpoint=os.environ["AZURE_AI_PROJECT_ENDPOINT"],
    credential=DefaultAzureCredential(),
)

# Create MCP tool
knowledge_mcp = MCPTool(server_label="salesforce-knowledge", server_url=f"http://127.0.0.1:{KB_PORT}/sse")

print(f"✅ MCP tool configured: {knowledge_mcp.server_label}")

In [ ]:
# Cell 4: Create OpenAI Client + Load System Prompt
from pathlib import Path as _Path

system_prompt_path = _Path("../agents/service/system_prompt.md")
system_prompt = system_prompt_path.read_text(encoding="utf-8")

MODEL = os.environ.get("AZURE_OPENAI_DEPLOYMENT", "gpt-4o")
client = project_client.get_openai_client()

print(f"OpenAI client ready — model: {MODEL}")

In [ ]:
# Cell 5: KB Query — "How do I reset a customer's API key?"
from IPython.display import Markdown, display

query = "How do I reset a customer's API key?"

response = client.responses.create(
    model=MODEL,
    instructions=system_prompt,
    input=query,
    tools=[knowledge_mcp],
)

print(f"Response ID: {response.id}")
for item in response.output:
    if item.type == "message":
        for content in item.content:
            if content.type == "output_text":
                display(Markdown(content.text))

In [ ]:
# Cell 6: Follow-up — "What about SSO configuration?"

followup = "What about SSO configuration?"

followup_response = client.responses.create(
    model=MODEL,
    instructions=system_prompt,
    input=followup,
    tools=[knowledge_mcp],
    previous_response_id=response.id,
)

print(f"Response ID: {followup_response.id}")
for item in followup_response.output:
    if item.type == "message":
        for content in item.content:
            if content.type == "output_text":
                display(Markdown(content.text))

In [ ]:
# Cell 7: Cleanup — terminate MCP server

kb_process.terminate()
kb_process.wait(timeout=5)
print(f"Knowledge MCP server terminated (PID: {kb_process.pid}).")

print("\nKB assistant session complete.")